In [29]:
import pandas as pd
import os, re
import csv
import numpy as np

In [30]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 11})

plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 300

In [32]:
# Metadata for specific SFC configurations
exp_ls = "0x1"


In [33]:
def print_headers(exp_type, exp_num, file):
    file_path = os.path.join("..\\data", exp_type, exp_num, file)

    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  
        print("header：", headers)
    return headers


#print_headers("fixed_rate", exp_ls, "bridge-pcm-new.csv")


In [34]:
def parse_seq(exp_type, exp_num, file, index=5):
    "Parse sequential data, specialized for the TX/RX/Latency experiments"
    tmp = []
    
    if file.startswith("latency.csv"):
        with open(os.path.join("..\\data", exp_type, exp_num, file)) as f:
            lines = f.readlines()[4:] if exp_type == "ONVM-nDPI-stats" else f.readlines()
            for i in range(len(lines)):
                try:
                    value = float(lines[i].strip()) / 1000
                except ValueError:
                    value= np.nan
                tmp.append(value)
    else:
        with open(os.path.join("..\\data", exp_type, exp_num, file)) as f:
            next(f)
            for l in f:
                try:
                    value = float(l.strip().split(",")[index])
                except ValueError:
                    value = np.nan
                tmp.append(value)
                
    return tmp

In [35]:
def parse_reg(exp_type, exp_num, file, stat_name):
    stat = []
    file_path = os.path.join("..\\data", exp_type, exp_num, file)

    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)
        try:
            index = headers.index(stat_name)
        except ValueError:
            print(f"Column {stat_name} not found")
            return stat
        for row in reader:
            if len(row) <= index: 
                continue  

            raw_value = row[index]
            try:
                value = float(raw_value)
            except ValueError:
                value = raw_value
            stat.append(value)


    return stat

In [36]:
bridge_L3MISS = parse_reg("ONVM-nDPI-stats", exp_ls,  "ndpi_stats-pcm_new.csv", "Socket 0-L3HIT")
print(len(bridge_L3MISS))

299


In [37]:
features = print_headers("ONVM-nDPI-stats", exp_ls,  "ndpi_stats-pcm_new.csv")

header： ['System-Date', 'System-Time', 'Socket 0-EXEC', 'Socket 0-IPC', 'Socket 0-FREQ', 'Socket 0-AFREQ', 'Socket 0-L3MISS', 'Socket 0-L2MISS', 'Socket 0-L3HIT', 'Socket 0-L2HIT', 'Socket 0-L3MPI', 'Socket 0-L2MPI', 'Socket 0-L3OCC', 'Socket 0-LMB', 'Socket 0-RMB', 'Socket 0-READ', 'Socket 0-WRITE', 'Socket 0-LOCAL', 'Socket 0-TEMP', 'Socket 0-INST', 'Socket 0-ACYC', 'Socket 0-TIME(ticks)', 'Socket 0-PhysIPC', 'Socket 0-PhysIPC%', 'Socket 0-INSTnom', 'Socket 0-INSTnom%', 'SKT0 Core C-State-C0res%', 'SKT0 Core C-State-C1res%', 'SKT0 Core C-State-C3res%', 'SKT0 Core C-State-C6res%', 'SKT0 Core C-State-C7res%', 'SKT0 Package C-State-C0res%', 'SKT0 Package C-State-C2res%', 'SKT0 Package C-State-C3res%', 'SKT0 Package C-State-C6res%', 'SKT0 Package C-State-C7res%', 'Proc Energy (Joules)-SKT0', 'DRAM Energy (Joules)-SKT0', 'LLCRDMISSLAT (ns)-SKT0', 'UncFREQ (Ghz)-SKT0', 'Core3 (Socket 0)-EXEC', 'Core3 (Socket 0)-IPC', 'Core3 (Socket 0)-FREQ', 'Core3 (Socket 0)-AFREQ', 'Core3 (Socket 0)-L3MI

In [38]:
features_pcie = print_headers("ONVM-nDPI-stats", exp_ls,   "pcm-pcie_new.csv")

header： ['skt-0_PCIRdCur', 'skt-0_RFO', 'skt-0_CRd', 'skt-0_DRd', 'skt-0_ItoM', 'skt-0_PRd', 'skt-0_WiL', 'skt-1_PCIRdCur', 'skt-1_RFO', 'skt-1_CRd', 'skt-1_DRd', 'skt-1_ItoM', 'skt-1_PRd', 'skt-1_WiL']


In [39]:
features_memory = print_headers("ONVM-nDPI-stats", exp_ls, "pcm-memory_new.csv")

header： ['SKT0-Ch0Read', 'SKT0-Ch0Write', 'SKT0-Ch1Read', 'SKT0-Ch1Write', 'SKT0-Ch2Read', 'SKT0-Ch2Write', 'SKT0-Ch3Read', 'SKT0-Ch3Write', 'SKT0-Mem Read (MB/s)', 'SKT0-Mem Write (MB/s)', 'SKT0-P. Write (T/s)', 'SKT0-Memory (MB/s)', 'System-Read', 'System-Write', 'System-Memory']


In [40]:
Infos=["ndpi_stats-pcm_new.csv"]
Infos_pcie = ["pcm-pcie_new.csv"]
Infos_memory =["pcm-memory_new.csv"]
def make_df():
    tmp = []
    dic = dict()

    seq_length = min(len(parse_reg("ONVM-nDPI-stats", exp_ls, "ndpi_stats-pcm_new.csv", 'System-Date')), 
                     len(parse_reg("ONVM-nDPI-stats", exp_ls, "pcm-pcie_new.csv", 'skt-0_PCIRdCur')))

    for f in features:
        for v in Infos:
            tmp =parse_reg("ONVM-nDPI-stats", exp_ls, "ndpi_stats-pcm_new.csv", f)
            tmp = tmp[:seq_length]
            if len(tmp) < seq_length:
                tmp = np.pad(tmp, (0, seq_length - len(tmp)), mode='constant')
            dic[v.split(".")[0] + "_" + f.replace(",", "")] = tmp
            
    for f in features_pcie:
        for v in Infos_pcie:
            tmp =parse_reg("ONVM-nDPI-stats", exp_ls, "pcm-pcie_new.csv", f)
            tmp = tmp[:seq_length]
            if len(tmp) < seq_length:
                tmp = np.pad(tmp, (0, seq_length - len(tmp)), mode='constant')
            dic[v.split(".")[0] + "_" + f.replace(",", "")] = tmp 
    for f in features_memory:
        for v in Infos_memory:
            tmp =parse_reg("ONVM-nDPI-stats", exp_ls, "pcm-memory_new.csv", f)
            tmp = tmp[:seq_length]
            if len(tmp) < seq_length:
                tmp = np.pad(tmp, (0, seq_length - len(tmp)), mode='constant')
            dic[v.split(".")[0] + "_" + f.replace(",", "")] = tmp 
    
    return pd.DataFrame(dic)

#bridge_L3MISS = parse_reg("fixed_rate", exp_ls,  "bridge-pcm.csv", "Socket 0-L3HIT")
#print(len(bridge_L3MISS))

In [41]:
df = make_df()


In [42]:
print(df)

    ndpi_stats-pcm_new_System-Date ndpi_stats-pcm_new_System-Time  \
0                       2023-11-28                   14:52:40.335   
1                       2023-11-28                   14:52:41.336   
2                       2023-11-28                   14:52:42.336   
3                       2023-11-28                   14:52:43.336   
4                       2023-11-28                   14:52:44.336   
..                             ...                            ...   
193                     2023-11-28                   14:55:53.356   
194                     2023-11-28                   14:55:54.357   
195                     2023-11-28                   14:55:55.357   
196                     2023-11-28                   14:55:56.357   
197                     2023-11-28                   14:55:57.357   

     ndpi_stats-pcm_new_Socket 0-EXEC  ndpi_stats-pcm_new_Socket 0-IPC  \
0                                0.32                             1.58   
1                      

In [27]:
abs_path = os.path.join("..\\datasets", "ONVM-nDPI-stats_"+ exp_ls +".csv")
df.to_csv(abs_path, sep=',')


In [28]:
for i in [1,10,10000,2,3,6,"f","fffff"]:
    exp_ls = f'0x{i}'
    print(exp_ls)
    df = make_df()
    abs_path = os.path.join("..\\datasets", "ONVM-nDPI-stats_"+ exp_ls +".csv")
    df.to_csv(abs_path, sep=',')

0x1
0x10
0x10000
0x2
0x3
0x6
0xf
0xfffff
